# Middleware Project: Big Data

## Initialization

In [1]:
DATASET_DIR = './dataset/'

In [2]:
# imports
import operator
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [3]:
# initialize spark
sc = pyspark.SparkContext("local[4]", "MiddlewareProject")
spark = SparkSession.builder.appName("MiddlewareProject").getOrCreate()

## test vari

In [4]:
# come aprire un file
rdd = spark.read.csv(DATASET_DIR + 'short.csv', header=True)

In [5]:
# roba con dataframe
rdd.first()

Row(Year='1994', Month='1', DayofMonth='7', DayOfWeek='5', DepTime='858', CRSDepTime='900', ArrTime='954', CRSArrTime='1003', UniqueCarrier='US', FlightNum='227', TailNum='NA', ActualElapsedTime='56', CRSElapsedTime='63', AirTime='NA', ArrDelay='-9', DepDelay='-2', Origin='CLT', Dest='ORF', Distance='290', TaxiIn='NA', TaxiOut='NA', Cancelled='0', CancellationCode='NA', Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')

In [6]:
rdd.count()

100000

In [7]:
# replace 'NA' with null values
rdd = rdd.replace('NA', None)

In [8]:
# find columns with null values
for col in rdd.columns:
    num = rdd.filter(rdd[col].isNull()).count()
    if num > 0:
        print("{:17} -> {}".format(col, num))

DepTime           -> 5754
ArrTime           -> 6135
TailNum           -> 100000
ActualElapsedTime -> 6135
AirTime           -> 100000
ArrDelay          -> 6135
DepDelay          -> 5754
TaxiIn            -> 100000
TaxiOut           -> 100000
CancellationCode  -> 100000
CarrierDelay      -> 100000
WeatherDelay      -> 100000
NASDelay          -> 100000
SecurityDelay     -> 100000
LateAircraftDelay -> 100000


In [9]:
# cast columns to correct type
intcolumns = ['Year', 'Month', 'DayofMonth', 'DayofWeek', 'DepTime', 'CRSDEPTime', 'ArrTime', 'CRSArrTime',
              'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Distance', 'CarrierDelay',
              'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Cancelled', 'Diverted']

for c in intcolumns:
    rdd = rdd.withColumn(c, rdd[c].cast('int'))

In [10]:
rdd.first()

Row(Year=1994, Month=1, DayofMonth=7, DayofWeek=5, DepTime=858, CRSDEPTime=900, ArrTime=954, CRSArrTime=1003, UniqueCarrier='US', FlightNum='227', TailNum=None, ActualElapsedTime=56, CRSElapsedTime=63, AirTime=None, ArrDelay=-9, DepDelay=-2, Origin='CLT', Dest='ORF', Distance=290, TaxiIn=None, TaxiOut=None, Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None)

In [11]:
# to pandas dataframe
df = rdd.toPandas()

In [21]:
df.head(5)

,Year,Month,DayofMonth,DayofWeek,DepTime,CRSDEPTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1994,1,7,5,858.0,900,954.0,1003,US,227,...,None,None,0,None,0,None,None,None,None,None
1,1994,1,8,6,859.0,900,952.0,1003,US,227,...,None,None,0,None,0,None,None,None,None,None
2,1994,1,10,1,935.0,900,1023.0,1003,US,227,...,None,None,0,None,0,None,None,None,None,None
3,1994,1,11,2,903.0,900,1131.0,1003,US,227,...,None,None,0,None,0,None,None,None,None,None
4,1994,1,12,3,933.0,900,1024.0,1003,US,227,...,None,None,0,None,0,None,None,None,None,None


In [12]:
df.describe()

,Year,Month,DayofMonth,DayofWeek,DepTime,CRSDEPTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Distance,Cancelled,Diverted
count,100000.0,100000.0,100000.000000,100000.000000,94246.000000,100000.000000,93865.000000,100000.000000,93865.000000,100000.00000,93865.000000,94246.000000,100000.000000,100000.000000,100000.000000
mean,1994.0,1.0,16.043110,3.961010,1375.857267,1358.403500,1499.445235,1486.779440,85.846876,87.33814,9.671305,11.378679,452.015090,0.057540,0.003810
std,0.0,0.0,8.889891,2.066788,451.669424,444.294523,455.402647,445.426795,47.624833,46.30459,26.917011,23.883334,354.019212,0.232873,0.061608
min,1994.0,1.0,1.000000,1.000000,1.000000,530.000000,1.000000,5.000000,15.000000,19.00000,-41.000000,-13.000000,32.000000,0.000000,0.000000
25%,1994.0,1.0,8.000000,2.000000,1000.000000,950.000000,1127.000000,1120.000000,56.000000,60.00000,-5.000000,0.000000,248.000000,0.000000,0.000000
50%,1994.0,1.0,16.000000,4.000000,1356.000000,1345.000000,1518.000000,1513.000000,71.000000,73.00000,2.000000,2.000000,335.000000,0.000000,0.000000
75%,1994.0,1.0,24.000000,6.000000,1744.000000,1730.000000,1902.000000,1853.000000,99.000000,100.00000,14.000000,13.000000,526.000000,0.000000,0.000000
max,1994.0,1.0,31.000000,7.000000,2400.000000,2359.000000,2400.000000,2400.000000,560.000000,370.00000,423.000000,434.000000,2521.000000,1.000000,1.000000


## Actual assignment

In [14]:
# the percentage of canceled flights per day

In [15]:
# weekly percentages of delays that are due to weather

In [16]:
# the percentage of flights belonging to a given "distance group"
# that were able to halve their departure delays by the time they 
# arrived at their destinations. 

# Distance groups assort flights by their total distance in miles.
# Flights with distances that are less than 200 miles belong in group 1,
# flights with distances that are between 200 and 399 miles belong 
# in group 2, flights with distances that are between 400 and 599 miles
# belong in group 3, and so on. The last group contains flights whose
# distances are between 2400 and 2599 miles.

In [17]:
# a weekly "penalty" score for each airport that depends on both 
# the its incoming and outgoing flights. 

# The score adds 0.5 for each incoming flight that is more than 15 minutes
# late, and 1 for each outgoing flight that is more than 15 minutes late

In [18]:
# an additional data analysis defined by your group

In [19]:
# prima fare con un solo file come debug, poi mettere tutti

In [20]:
# come caricare più file allo stesso tempo: